In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
# 주가 데이터 불러오기
apple = yf.download('AAPL', start='2010-01-01', end='2025-02-01')


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
apple.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.440331,6.455077,6.391278,6.422877,493729600
2010-01-05,6.451466,6.487879,6.417459,6.458086,601904800
2010-01-06,6.348846,6.477045,6.342225,6.451465,552160000
2010-01-07,6.337112,6.379845,6.291069,6.372322,477131200
2010-01-08,6.379241,6.379843,6.291368,6.328684,447610800


## 재무제표

**변수명 설명**
- rcept_no : 접수번호 (전자공시시스템 상의 공시문서 번호)
- reprt_code : 보고서 코드
- bsns_year :	사업연도
- corp_code :	기업 고유번호(DART에서 기업 구분에 사용)
- sj_div : 재무제표 구분 코드 (BS: 재무상태표, IS: 손익계산서, CF: 현금흐름표)
- sj_nm :	재무제표명(한글)
- account_id : 계정 항목 IFRS 표준 코드 (국제회계기준)
- account_nm : 계정 항목 이름(자산총계, 현금및현금성자산, 순이익 등)
- account_detail : 세부 계정 항목 (대부분 상세 설명 없을 때 비어있음)
- thstrm_nm :	당기 명칭 (최근 결산된 사업 연도 명칭)
- thstrm_amount :	당기 금액 (해당 보고서의 최근 결산 금액, 당해년도)
- frmtrm_nm :	전기 명칭 (당기 직전 연도)
- frmtrm_amount :	전기 금액 (직전 연도의 결산 금액)
- bfefrmtrm_amount : 전전기 금액 (2년 전 결산 금액)
- bsns_year :	사업연도
- corp_code :	기업 고유번호
- fs_div : 재무제표 구분 (CFS: 연결 재무제표, OFS: 별도 재무제표)
- ord : 재무제표 내에서 해당 항목의 순서(위치)
- account_detail : 세부계정 항목(보통 추가적인 설명이 없으면 빈칸임)
- thstrm_add_amount : 추가 금액(별도 항목이 있을 때 제공, 보통 NaN)

In [12]:
import OpenDartReader
import pandas as pd

# API Key 설정
api_key = 'c54fb16d5d02248a619b7e46a0b996893bf2408d'

# OpenDartReader 객체 생성
dart = OpenDartReader(api_key)

# 기업 고유번호 가져오기 (삼성전자 예시)
corp_code = dart.find_corp_code('삼성전자')

# 특정년도(2023년)의 연결 재무제표 (재무상태표, 손익계산서, 현금흐름표 등) 가져오기
fs = dart.finstate_all(corp_code, 2023, reprt_code='11011', fs_div='CFS')

# 결과 확인
fs.head()



reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'


,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20240312000736,11011,2023,00126380,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 55 기,455905980000000,제 54 기,448424507000000,제 53 기,426621158000000,7,KRW,NaN
1,20240312000736,11011,2023,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 55 기,195936557000000,제 54 기,218470581000000,제 53 기,218163185000000,8,KRW,NaN
2,20240312000736,11011,2023,00126380,BS,재무상태표,dart_ShortTermOtherReceivables,미수금,-,제 55 기,6633248000000,제 54 기,6149209000000,제 53 기,4497257000000,9,KRW,NaN
3,20240312000736,11011,2023,00126380,BS,재무상태표,dart_ShortTermPrepaidExpenses,선급비용,-,제 55 기,3366130000000,제 54 기,2867823000000,제 53 기,2336252000000,10,KRW,NaN
4,20240312000736,11011,2023,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 55 기,69080893000000,제 54 기,49680710000000,제 53 기,39031415000000,11,KRW,NaN


In [11]:
# 원하는 항목만 선택하여 보기 쉽게 정리 (예시: 손익계산서 항목만)
income_statement = fs[fs['sj_nm'] == '손익계산서']

# 손익계산서에서 주요 항목만 선택 (매출액, 영업이익, 당기순이익 등)
selected_items = income_statement[income_statement['account_nm'].isin(['매출액', '영업이익', '당기순이익'])]

selected_items[['account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']]


,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
52,영업이익,6566976000000,43376630000000,51633856000000


In [8]:
fs['change_rate'] = (fs['thstrm_amount'].astype(float) - fs['frmtrm_amount'].astype(float)) / fs['frmtrm_amount'].astype(float)


ValueError: could not convert string to float: ''